In [ ]:
import tensorflow as tf
from tensorflow.keras import layers,models
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from tensorflow.keras.layers import LeakyReLU

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
images = np.load('/content/drive/MyDrive/Data/images10.npy')
labels = np.load('/content/drive/MyDrive/Data/labels10.npy')

In [ ]:



encoder = OneHotEncoder()
labels_one_hot = encoder.fit_transform(labels.reshape(-1, 1)).toarray()

height, width, channels = images.shape[1], images.shape[2], images.shape[3]
num_classes = labels_one_hot.shape[1]



model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.1), input_shape = (height, width, channels)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.1)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation=tf.keras.layers.LeakyReLU(alpha=0.1)),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(images, labels_one_hot, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

history = model.fit(X_train, y_train, epochs=16, validation_data=(X_val, y_val))

test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

model.save('terrain_recognition_model.h5')


Epoch 1/16
13/13 [==============================] - 12s 55ms/step - loss: 1.1215 - accuracy: 0.5224 - val_loss: 0.7274 - val_accuracy: 0.6860
Epoch 2/16
13/13 [==============================] - 0s 10ms/step - loss: 0.7762 - accuracy: 0.6990 - val_loss: 0.5622 - val_accuracy: 0.7209
Epoch 3/16
13/13 [==============================] - 0s 10ms/step - loss: 0.5819 - accuracy: 0.7587 - val_loss: 0.3857 - val_accuracy: 0.8372
Epoch 4/16
13/13 [==============================] - 0s 10ms/step - loss: 0.4658 - accuracy: 0.8259 - val_loss: 0.4351 - val_accuracy: 0.8023
Epoch 5/16
13/13 [==============================] - 0s 10ms/step - loss: 0.4203 - accuracy: 0.8209 - val_loss: 0.2683 - val_accuracy: 0.8605
Epoch 6/16
13/13 [==============================] - 0s 9ms/step - loss: 0.4701 - accuracy: 0.7960 - val_loss: 0.3925 - val_accuracy: 0.8372
Epoch 7/16
13/13 [==============================] - 0s 10ms/step - loss: 0.3677 - accuracy: 0.8532 - val_loss: 0.3792 - val_accuracy: 0.8256
Epoch 8/16
13

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
pip install mahotas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 18.4 MB/s eta 0:00:00


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import mahotas.features
from google.colab.patches import cv2_imshow

# Load the pre-trained CNN model
cnn_model = load_model('/content/terrain_recognition_model.h5')

# Function to extract Haralick texture features (contrast, energy, entropy)
def extract_haralick_features(image):
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Calculate Haralick texture features
    haralick_features = mahotas.features.haralick(gray_image)

    # Calculate contrast, energy, and entropy
    contrast = haralick_features[:, 1].mean()
    energy = haralick_features[:, 2].mean()
    entropy = haralick_features[:, 8].mean()

    return contrast, energy, entropy

# Load and preprocess the image you want to classify
image_to_classify = cv2.imread('/content/drive/MyDrive/Data/cropped (8).jpg')
image_to_classify =cv2.resize(image_to_classify,(64,64))


# Extract Haralick texture features from the image
contrast, energy, entropy = extract_haralick_features(image_to_classify)

# Print the extracted features


# Assuming the CNN model expects an input shape of (64, 64, 3)
# You may need to adjust the preprocessing based on your specific CNN model
cnn_input = np.expand_dims(image_to_classify, axis=0)  # Expand dimensions to match the CNN input shape

# Make predictions using the CNN model
predicted_class_probabilities = cnn_model.predict(cnn_input)

# Assuming the CNN model outputs class probabilities, get the predicted class index
predicted_class_index = np.argmax(predicted_class_probabilities)

# Define a mapping of CNN model class labels to actual labels


if(predicted_class_index>=2):

  # Load the grayscale image
  image = cv2.imread('/content/drive/MyDrive/Data/cropped (8).jpg', cv2.IMREAD_GRAYSCALE)
  image = cv2.resize(image, (64, 64))
  hist, bins = np.histogram(image.flatten(), 256, [0, 256])
  hist_prob = hist.astype(float) / np.sum(hist)
  max_variance = 0
  threshold = 0
  for i in range(1, 256):
    w0 = np.sum(hist_prob[:i])  # Probability of class 0 (background)
    w1 = np.sum(hist_prob[i:])  # Probability of class 1 (foreground)

    mean0 = np.sum(hist_prob[:i] * np.arange(i)) / (w0 + 1e-10)  # Mean of class 0
    mean1 = np.sum(hist_prob[i:] * np.arange(i, 256)) / (w1 + 1e-10)  # Mean of class 1

    # Calculate between-class variance
    variance_between = w0 * w1 * (mean0 - mean1) ** 2

    # Update threshold if variance is greater
    if variance_between > max_variance:
        max_variance = variance_between
        threshold = i
    # Apply the optimal threshold to the image
  binary_image = np.where(image >= threshold, 255, 0).astype(np.uint8)

  if(threshold>136):
        print("Rocky type")


  else:

    #class_mapping_cnn={3: 'class_3'}
    #print('Predicted Class (CNN):', class_mapping_cnn[predicted_class_index])
    print("Sand type")

# Print the predicted class and its associated probability
else:

  class_mapping_cnn={0:'Grass type' ,1:'Marshy type' }
  print('Predicted Class (CNN):', class_mapping_cnn[predicted_class_index])
  print('Predicted Class Probability (CNN):', predicted_class_probabilities[0, predicted_class_index])




1/1 [==============================] - 0s 237ms/step
Rocky type
